<a href="https://colab.research.google.com/github/MabelKrugerDeSouza/SearchDocumentInEmbeddingAI/blob/main/searchDocumentWithEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U google-generativeai

In [51]:
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata
api_key = userdata.get('key')
genai.configure(api_key=api_key)

# Code

In [54]:
# So transformando um titulo e uma descricao em embedding
title = "Tempo"
sample_text = ("Titulo: Final de semana de sol no vale do itajai"
  "\n"
  "Mas no proxima semana temperaturas despencam e o frio começa a aparecer"
)

embeddings = genai.embed_content(
    model= "models/embedding-001",
    content= sample_text,
    title = title,
    task_type="RETRIEVAL_DOCUMENT"
    )

print(embeddings)

{'embedding': [0.023772847, 0.010472972, -0.024390956, 0.031977337, 0.064187005, 0.006179242, 0.007613137, -0.035756912, 0.017418599, 0.07558835, -0.026193213, 0.0018877942, 0.0061977957, -0.0061834054, -0.046432093, -0.004098101, -0.029778974, 0.049388114, -0.0021095322, -0.03794975, 0.012575654, 0.019950224, -0.034825657, -0.04137452, 0.011658829, 0.0015323911, 0.03343949, -0.019567257, -0.0053940364, 0.032225437, -0.018095326, 0.057625346, -0.022198962, 0.0001239071, -0.015639346, -0.023292135, -0.044560906, 0.020508835, 0.011070075, 0.04311478, 0.0071884156, -0.054144483, -0.027211264, 0.03160921, 0.023593636, -0.01561955, 0.0054912106, -0.005260909, 0.012108387, -0.02967641, 0.042106032, -0.030906178, 0.086483814, -0.0072282413, -0.019373393, -0.0107637495, 0.042695418, -0.0033573876, 0.010837029, -0.006937029, -0.025209824, 0.008506487, 0.04108943, 0.039207116, -0.039014664, -0.04803838, -0.05229919, 0.004134769, 0.023726404, 0.0014914632, 0.025125748, -0.050339036, 0.05703901, 0

In [53]:
#Criacao de uma lista de "documentos" para usar mais pra frente
DOCUMENT1 = {
    "Titulo": "Teste1",
    "Conteúdo": "Testanto o teste 1"
}

DOCUMENT2 = {
    "Titulo": "Teste2",
    "Conteúdo": "Testanto o teste 2"
}

DOCUMENT3 = {
    "Titulo": "Teste3",
    "Conteúdo": "Testanto o teste 3"
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [40]:
df = pd.DataFrame(documents)
df.columns = ["Titúlo", "Conteúdos"]
df

,Titúlo,Conteúdos
0,Teste1,Testanto o teste 1
1,Teste2,Testanto o teste 2
2,Teste3,Testanto o teste 3


In [15]:
model = "models/embedding-001"
task_type = "RETRIEVAL_DOCUMENT"

In [18]:
def embed_fn(title, text):
  return genai.embed_content(
    model= model,
    content= text,
    title = title,
    task_type= task_type
    )["embedding"]



In [43]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titúlo"], row["Conteúdos"]), axis=1)
df

,Titúlo,Conteúdos,Embeddings
0,Teste1,Testanto o teste 1,"[0.0045748916, -0.02768944, -0.043641165, -0.0..."
1,Teste2,Testanto o teste 2,"[0.00026943337, -0.03436048, -0.042931378, 0.0..."
2,Teste3,Testanto o teste 3,"[0.019457709, -0.027618358, -0.041774254, -0.0..."


In [41]:
task_type2 = "RETRIEVAL_QUERY"


def generate_search_and_consult(consult, base, model):

  # Está pegando a consulta e transoformando em embedding
  embedding_of_consult = genai.embed_content(model=model,
                                 content=consult,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  # Aqui está fazendo o calculo de distancia entre a consulta e os documentos, pra ver qual esta mais proxima
  products_dot = np.dot(np.stack(df["Embeddings"]), embedding_of_consult)

  # Mostra qual o indice que está a consulta, se quiser mostra o conteudo do indice, so colocar assim
  # i = np.argmax(products_dot)["Contéudos"] -> conteudos está o nome da coluna, pode ser qualquer um
  indice = np.argmax(products_dot)
  return df.iloc[indice]["Conteúdos"]

In [44]:
# Aqui estamos fazendo a consulta de fato
consult = "o teste 2"

trecho = generate_search_and_consult(consult, df, model)
print(trecho)

Testanto o teste 2


In [45]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [49]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

Bora testar esse teste!
